# Credit Risk Resampling Techniques

In [141]:
import warnings
warnings.filterwarnings('ignore')

In [142]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [143]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [144]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [146]:
# Create our features
# create features excluding loan_status which is our target
columns_interest = [i for i in df.columns if i not in ('loan_status')]
X = df.copy()
X = X[columns_interest]

# Create our target
y = df.copy()
y.drop(columns_interest, axis='columns', inplace=True)


In [147]:
#get the columns whose datatype is object to encode
column_name = X.columns
object_columns = []
for i in column_name:
    if X[i].dtype == 'object':
        print(f"column: {i}, datatype: {X[i].dtype}")
        object_columns.append(i)


column: home_ownership, datatype: object
column: verification_status, datatype: object
column: issue_d, datatype: object
column: pymnt_plan, datatype: object
column: initial_list_status, datatype: object
column: next_pymnt_d, datatype: object
column: application_type, datatype: object
column: hardship_flag, datatype: object
column: debt_settlement_flag, datatype: object


In [148]:
#encode using pandas get_dummies
X = pd.get_dummies(X, columns = object_columns)

In [149]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [150]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [233]:
# Create X_train, X_test, y_train, y_test
#imprt train_test_split dependency from sklearn kit
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5)
Counter(y_train['loan_status'])

Counter({'low_risk': 51355, 'high_risk': 257})

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [234]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled['loan_status'])

Counter({'low_risk': 51355, 'high_risk': 51355})

In [235]:
#import Logistic Regression dependency
from sklearn.linear_model import LogisticRegression
#Create Logistic Regression and train using the resampled data (RandomOverSampler)
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [236]:
#using trained model predict the loan_status
predictions = model.predict(X_test)
#calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, predictions)

0.6772616613107411

In [274]:
#display the confusion_matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,64,26
Actual 1,6103,11012


In [238]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.71      0.64      0.02      0.68      0.46        90
   low_risk       1.00      0.64      0.71      0.78      0.68      0.45     17115

avg / total       0.99      0.64      0.71      0.78      0.68      0.45     17205



### SMOTE Oversampling

In [239]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_smote_resampled, y_smote_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train, y_train)
Counter(y_smote_resampled['loan_status'])

Counter({'low_risk': 51355, 'high_risk': 51355})

In [242]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
smote_model = LogisticRegression(solver='lbfgs', random_state=1)
smote_model.fit(X_smote_resampled, y_smote_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [243]:
#using smote model predict value
smote_predictions = smote_model.predict(X_test)
#Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, smote_predictions)

0.6792336157366832

In [275]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, smote_predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,61,29
Actual 1,5465,11650


In [246]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, smote_predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.68      0.68      0.02      0.68      0.46        90
   low_risk       1.00      0.68      0.68      0.81      0.68      0.46     17115

avg / total       0.99      0.68      0.68      0.81      0.68      0.46     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the following steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [248]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cluster_centroids = ClusterCentroids(random_state=1)
X_cc_resampled, y_cc_resampled = cluster_centroids.fit_resample(X_train, y_train)
Counter(y_cc_resampled['loan_status'])

Counter({'high_risk': 257, 'low_risk': 257})

In [249]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
cc_model = LogisticRegression(solver='lbfgs', random_state=1)
cc_model.fit(X_cc_resampled, y_cc_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [250]:
#using cluster_centroids model predict 
cc_predictions = cc_model.predict(X_test)
#Calculate the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, cc_predictions)

0.5687798227675528

In [276]:
#Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, cc_predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,62,28
Actual 1,9436,7679


In [252]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, cc_predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.69      0.45      0.01      0.56      0.32        90
   low_risk       1.00      0.45      0.69      0.62      0.56      0.30     17115

avg / total       0.99      0.45      0.69      0.62      0.56      0.30     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the following steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [258]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
smote_een = SMOTEENN(random_state=1)
X_smoteen_resampled, y_smoteen_resampled = smote_een.fit_resample(X_train, y_train)
Counter(y_smoteen_resampled['loan_status'])

Counter({'high_risk': 51341, 'low_risk': 46476})

In [259]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
smoteen_model = LogisticRegression(solver='lbfgs', random_state=1)
smoteen_model.fit(X_smoteen_resampled, y_smoteen_resampled)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=1, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [260]:
#using smoteen model predict the values
smoteen_predictions = smoteen_model.predict(X_test)
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, smoteen_predictions)

0.6880173337228552

In [277]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, smoteen_predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,71,19
Actual 1,7066,10049


In [257]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, smoteen_predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.79      0.59      0.02      0.68      0.47        90
   low_risk       1.00      0.59      0.79      0.74      0.68      0.45     17115

avg / total       0.99      0.59      0.79      0.74      0.68      0.45     17205



# Extension

In [282]:
from sklearn.preprocessing import StandardScaler
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [283]:
#using ensemble algorithm predict credit risks of loan applications
#import dependency for BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brfc_model = BalancedRandomForestClassifier(n_estimators=100, random_state=0)
brfc_model.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                               criterion='gini', max_depth=None,
                               max_features='auto', max_leaf_nodes=None,
                               max_samples=None, min_impurity_decrease=0.0,
                               min_samples_leaf=2, min_samples_split=2,
                               min_weight_fraction_leaf=0.0, n_estimators=100,
                               n_jobs=None, oob_score=False, random_state=0,
                               replacement=False, sampling_strategy='auto',
                               verbose=0, warm_start=False)

In [284]:
#predict values using the BalancedRandomForestClassifier model
brfc_predictions = brfc_model.predict(X_test_scaled)
#calculate the balanced accuracy score
balanced_accuracy_score(y_test, brfc_predictions)

0.7759518940500536

In [285]:
#calculate the confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, brfc_predictions)

# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,59,31
Actual 1,1774,15341


In [286]:
#calculate the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, brfc_predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.66      0.90      0.06      0.77      0.57        90
   low_risk       1.00      0.90      0.66      0.94      0.77      0.60     17115

avg / total       0.99      0.90      0.66      0.94      0.77      0.60     17205



In [289]:
#get the feature importance
feature_importances = brfc_model.feature_importances_
# We can sort the features by their importance.
sorted(zip(feature_importances, X.columns), reverse=True)

[(0.07762983622975046, 'total_rec_prncp'),
 (0.07579788685332024, 'total_pymnt_inv'),
 (0.05697857749920097, 'total_rec_int'),
 (0.053949194431681516, 'total_pymnt'),
 (0.05199383708770626, 'last_pymnt_amnt'),
 (0.02951900655241165, 'int_rate'),
 (0.023514224882327177, 'issue_d_Jan-2019'),
 (0.019152114056733564, 'dti'),
 (0.01833788496396576, 'il_util'),
 (0.01784863941262188, 'installment'),
 (0.01649741475567708, 'out_prncp'),
 (0.01585451909361621, 'revol_bal'),
 (0.015770334645788288, 'out_prncp_inv'),
 (0.01558858522291501, 'mths_since_recent_inq'),
 (0.015473112796933374, 'tot_cur_bal'),
 (0.015086461349526965, 'annual_inc'),
 (0.01493414669356879, 'mo_sin_old_rev_tl_op'),
 (0.014696959518644715, 'bc_util'),
 (0.014490457072220592, 'mo_sin_old_il_acct'),
 (0.014455442995858356, 'bc_open_to_buy'),
 (0.014100019349291918, 'issue_d_Mar-2019'),
 (0.013946514853859613, 'avg_cur_bal'),
 (0.013341499217816865, 'max_bal_bc'),
 (0.013303353778052664, 'total_rev_hi_lim'),
 (0.013035552878

In [291]:
#Classifying using EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
eec_model = EasyEnsembleClassifier(n_estimators=100, random_state=0)
eec_model.fit(X_train_scaled, y_train)


EasyEnsembleClassifier(base_estimator=None, n_estimators=100, n_jobs=None,
                       random_state=0, replacement=False,
                       sampling_strategy='auto', verbose=0, warm_start=False)

In [292]:
#predict using the model defined for EasyEnsembleClassifier
eec_predictions = eec_model.predict(X_test_scaled)
#calculate the balanced accuracy score
balanced_accuracy_score(y_test, eec_predictions)

0.9225435777583018

In [293]:
#calculate confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, eec_predictions)


array([[   82,     8],
       [ 1130, 15985]])

In [294]:
#calculate imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, eec_predictions))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.07      0.91      0.93      0.13      0.92      0.85        90
   low_risk       1.00      0.93      0.91      0.97      0.92      0.85     17115

avg / total       0.99      0.93      0.91      0.96      0.92      0.85     17205

